In [5]:
import math

def ratings_to_size(rating):
    return (math.sqrt(rating) + 20) / 10

In [6]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Get the database credentials from environment variables
user = os.getenv("MYSQL_USER")
password = os.getenv("MYSQL_PASSWORD")
host = os.getenv("MYSQL_HOST")
database = os.getenv("MYSQL_DATABASE")

# Create an SQLAlchemy engine for the connection
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# Query to retrieve latitude and longitude
query = "SELECT latitude, longitude FROM address WHERE location_type = 'ROOFTOP';"

# Load data into a pandas DataFrame
raw_df = pd.read_sql(query, engine)

patient_df = raw_df

# Display the data
patient_df.head()



,latitude,longitude
0,-36.7875,174.610
1,-36.8152,174.635
2,-36.8385,174.627
3,-36.8337,174.604
4,-36.8430,174.612


In [7]:
import pickle 
# Load the property coordinates from the Parquet file
property_data = pd.read_parquet('data/nz_property_lat_lon.parquet')
property_data = property_data.drop(columns=['WKT'])

# Load the deduplicated dentists
with open('data/deduplicated_dentists.pkl', 'rb') as f:
    deduplicated_dentists = pickle.load(f)

# Convert deduplicated_dentists to DataFrame for easier manipulation
dentists_df = pd.DataFrame(deduplicated_dentists)
dentists_df['size'] = dentists_df['reviews'].apply(ratings_to_size)

# Display a few rows to verify
display(property_data.head())
display(dentists_df.head())


,longitude,latitude
0,172.682428,-43.569101
1,174.766058,-36.850536
2,174.766289,-36.848008
3,174.770643,-36.847417
4,174.753391,-36.860142


,place_id,name,lat,lon,reviews,size
0,ChIJn2iTgIGqEm0R3tcUFgH1VDY,Mint Dental,-37.191588,174.903652,39,2.624500
1,ChIJZ1N2dWKqEm0RZV_x3mljNxU,Pukekohe Orthodontists,-37.074346,174.922603,3,2.173205
2,ChIJPx17MdWscm0RaYxnKNKfAcg,Vanessa Wright Dental,-37.062908,174.940487,11,2.331662
3,ChIJiQWVRdSscm0RgB5b7NnhmNM,The Denture Man,-37.064653,174.943538,9,2.300000
4,ChIJz13Dp9Wscm0Ru9TUvS4DslY,Dental World Papakura,-37.063301,174.943527,7,2.264575


In [8]:
import numpy as np

# Function to calculate distance between two points (Haversine formula)
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r  # Distance in kilometers

# Test the function
print(haversine(-36.8485, 174.7633, -36.8484, 174.7634))  # Test distance in Auckland


0.014241458474257587


In [9]:
# Locate the row corresponding to Massey Smiles in the dentists_df DataFrame
massey_smiles_row = dentists_df[dentists_df['name'].str.contains('Massey Smiles', case=False)]

# Extract the latitude and longitude for Massey Smiles
massey_smiles_lat = massey_smiles_row['lat'].values[0]
massey_smiles_lon = massey_smiles_row['lon'].values[0]
massey_smiles_size = ratings_to_size(massey_smiles_row['reviews'].values[0])

# Print the extracted location to verify
print(f"Massey Smiles Location: Latitude = {massey_smiles_lat}, Longitude = {massey_smiles_lon} Size = {massey_smiles_size}")


Massey Smiles Location: Latitude = -36.82235440000001, Longitude = 174.6081585 Size = 2.8717797887081344


In [10]:
# Massey Smiles' location (replace with actual latitude and longitude)

# Calculate the distance of each patient to Massey Smiles using the Haversine formula
patient_df['distance_to_massey_smiles'] = haversine(
    patient_df['latitude'], patient_df['longitude'], massey_smiles_lat, massey_smiles_lon
)

# Display the first few rows to verify
display(patient_df[['latitude', 'longitude', 'distance_to_massey_smiles']].head())


   latitude  longitude  distance_to_massey_smiles
0  -36.7875    174.610                   3.879099
1  -36.8152    174.635                   2.518266
2  -36.8385    174.627                   2.456669
3  -36.8337    174.604                   1.314747
4  -36.8430    174.612                   2.321005


In [11]:
# Define distance brackets (e.g., 0-1 km, 1-2 km, etc.)
distance_brackets = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # You can adjust these as needed

# Bin the patients based on their distance to Massey Smiles
patient_df['distance_bracket'] = pd.cut(patient_df['distance_to_massey_smiles'], bins=distance_brackets)

# Count the number of patients in each distance bracket
actual_patient_distribution = patient_df['distance_bracket'].value_counts().sort_index()

# Display the actual patient distribution by distance

total_actual_patients = actual_patient_distribution.sum()
actual_percentage_distribution = (actual_patient_distribution / total_actual_patients) * 100
display(actual_percentage_distribution)

distance_bracket
(0, 1]      7.171686
(1, 2]     20.987271
(2, 3]     21.639242
(3, 4]     15.212667
(4, 5]      7.420056
(5, 6]      8.351444
(6, 7]      6.085067
(7, 8]      5.650419
(8, 9]      4.750078
(9, 10]     2.732071
Name: count, dtype: float64

In [12]:
display(actual_patient_distribution)

distance_bracket
(0, 1]     231
(1, 2]     676
(2, 3]     697
(3, 4]     490
(4, 5]     239
(5, 6]     269
(6, 7]     196
(7, 8]     182
(8, 9]     153
(9, 10]     88
Name: count, dtype: int64

In [13]:
import math

property_lats = property_data['latitude'].values
property_lons = property_data['longitude'].values
dentist_sizes = dentists_df['size']

In [14]:
# Define a function to predict the patient percentage distribution using the Huff model for Massey Smiles
def predict_patient_percentage_distribution(beta):
    # Calculate distances from all properties to Massey Smiles
    distances = haversine(property_lats, property_lons, massey_smiles_lat, massey_smiles_lon)

    # Calculate size-over-distance for each property
    size_over_distance = massey_smiles_size / (distances ** beta)

    # Normalize the size-over-distance values to get probabilities
    total_size_over_distance = size_over_distance.sum()
    if total_size_over_distance > 0:
        normalized_probabilities = size_over_distance / total_size_over_distance
    else:
        normalized_probabilities = np.zeros_like(size_over_distance)

    # Group properties into distance brackets and calculate the percentage of patients in each bracket
    predicted_percentages = []
    for bracket_start, bracket_end in zip(distance_brackets[:-1], distance_brackets[1:]):
        # Mask distances within the current bracket
        mask = (distances <= bracket_end) & (distances > bracket_start)
        
        # Sum the normalized probabilities for properties in the current bracket
        bracket_probability_sum = normalized_probabilities[mask].sum()

        # Append the percentage (between 0 and 1)
        predicted_percentages.append(bracket_probability_sum)

    # Convert to percentage (between 0 and 100)
    predicted_percentages = np.array(predicted_percentages) * 100

    return predicted_percentages

# Example: Predict the distribution with a beta value of 2.0
beta_test = 2.0
predicted_percentage_distribution = predict_patient_percentage_distribution(beta_test)

# Print the predicted percentage distribution
print("Predicted patient percentage distribution:", predicted_percentage_distribution)

# Check the sum of the percentages to ensure it adds to 100
print("Sum of predicted percentages:", predicted_percentage_distribution.sum())


Predicted patient percentage distribution: [9.80873626e+01 6.23736872e-01 2.39715604e-01 1.56317014e-01
 1.33529296e-01 9.79799690e-02 5.40508246e-02 5.24418200e-02
 4.24474589e-02 4.20120449e-02]
Sum of predicted percentages: 99.52959348915832


In [15]:
# Function to calculate error between actual and predicted distribution
def calculate_error(actual, predicted):
    return np.sum((actual - predicted) ** 2)


In [17]:
import numpy as np
from tqdm import tqdm

# Tune beta between a lower range to minimize the error
best_beta = None
best_error = float('inf')

# Iterate over a lower range of beta values
for beta in tqdm(np.arange(0.1, 2.0, 0.1), desc="Tuning Beta"):
    predicted_percentage_distribution = predict_patient_percentage_distribution(beta)
    error = calculate_error(actual_percentage_distribution.values, predicted_percentage_distribution)

    if error < best_error:
        best_beta = beta
        best_error = error

# Output the best beta and minimum error
print(f"Best Beta (Lowered Range): {best_beta}")
print(f"Minimum Error (Lowered Range): {best_error}")

# Print actual vs predicted for visual comparison
print("\nActual Patient Percentage Distribution:", actual_percentage_distribution.values)
print("Predicted Patient Percentage Distribution (Best Beta):", predict_patient_percentage_distribution(best_beta))


Tuning Beta: 100%|██████████| 19/19 [00:05<00:00,  3.44it/s]


Best Beta (Lowered Range): 1.1
Minimum Error (Lowered Range): 689.4772602224191

Actual Patient Percentage Distribution: [ 7.17168581 20.98727103 21.63924247 15.21266687  7.42005588  8.35144365
  6.08506675  5.65041912  4.75007762  2.73207079]
Predicted Patient Percentage Distribution (Best Beta): [11.86062873  7.28166417  4.44569158  4.00902829  4.29957567  3.77457872
  2.4050313   2.6780448   2.41804964  2.65088208]
